##### each_all.csv 파일은 23년 1월~12월 까지의 따릉이 이용정보 마스터 파일을 합친 파일 입니다.

In [1]:
import time
import datetime
import pandas as pd
import numpy as np
import typing as tp
import re
import matplotlib.pyplot as plt
import folium
import requests
import os
import gc
import seaborn as sns
# 필터링 용 커스텀 라이브러리
import MiniP as mp

# year data

In [2]:
# 최적화 함수 정의
def optimize_dataframe(df):
    # 숫자형 컬럼의 데이터 타입을 다운캐스팅
    for col in df.select_dtypes(include=['int']).columns:
        df[col] = pd.to_numeric(df[col], downcast='unsigned')
    for col in df.select_dtypes(include=['float']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    # 범주형 데이터로 변환
    for col in df.select_dtypes(include=['object']).columns:
        num_unique_values = len(df[col].unique())
        num_total_values = len(df[col])
        if num_unique_values / num_total_values < 0.5:
            df[col] = df[col].astype('category')
    return df
#
# 청크 단위로 데이터 로드 및 최적화
chunk_size = 100000  # 청크 사이즈 설정
chunks = []
for chunk in pd.read_csv('data/all_each.csv', chunksize=chunk_size):
    chunk_optimized = optimize_dataframe(chunk)
    chunks.append(chunk_optimized)
    del chunk, chunk_optimized
    gc.collect()
#
# 모든 청크를 결합
df_optimized = pd.concat(chunks, ignore_index=True)
year_data_df_all = df_optimized
del df_optimized, chunks
gc.collect()
filename1 = r"data/rental_id.csv" #파일명 작성
data = pd.read_csv(filename1,encoding='cp949')       
rental_id_df_all = pd.DataFrame(data)
year_data_df_sel = year_data_df_all[['대여일시','이용시간(분)','이용거리(M)','생년','성별','대여대여소ID','반납대여소ID']]
year_data_df_sel.columns = ['StartTime','UseTimeM','UseDistM','Birth','Gender','StartId','EndId']
rental_id_df_sel = rental_id_df_all[['대여소_ID', '주소1', '주소2','위도', '경도']]
rental_id_df_sel.columns = ['Id','Addr1','Addr2','Lat','Lon']

# 필터링 라이브러리

In [3]:
# 만일 각각의 필터링에 이상 사항이 생기면 직접수정 또는 오버라이딩을 통해 라이브러리를 수정하여 재배포 해주세요
year = mp.Filter(year_data_df_sel)
year_df = year.nan_filter
year_df = year.age_filter
year_df = year.datetime_filter
year_df = year.gender_filter
year_df = year.rental_id_filter
del year_data_df_sel
gc.collect()
rental_id = mp.Filter(rental_id_df_sel)
rental_id_df = rental_id.nan_filter
rental_id_df = rental_id.rental_id_filter
del rental_id_df_sel
gc.collect()
# 청크 사이즈 설정
chunk_size = 100000
# 첫 번째 파일을 청크 단위로 읽고 병합
chunks = []
# 2중 청크 반복문 
for i in range(0, len(year_df), chunk_size):
    chunk1 = year_df[i:i + chunk_size]
    chunk1 = optimize_dataframe(chunk1)
    for k in range(0, len(rental_id_df), chunk_size):
        chunk2 = rental_id_df[k:k + chunk_size]
        chunk2 = optimize_dataframe(chunk2)
        merged_chunk = pd.merge(chunk1, chunk2, left_on='StartId', right_on='Id', how='left')
        chunks.append(merged_chunk)
big_df = pd.concat(chunks, ignore_index=True)


NameError: name 'np' is not defined

##### 아래 코드는 실행하지 마시오
###### GeoPy 를 이용해 위도,경도 값을 주소로 받아오는 부분
###### 변환 시간이 50 분 정도 임으로 따로 CSV 파일로 저장 후 사용 중

In [ ]:
# #
# # 대여소 id 데이터의 위도 경도 값을 받아와 주소를 반환하는 기능
# #
# rental_id_df_sel_id = rental_id_df_sel['Id'].astype(str).tolist()
# rental_id_df_sel_lat = rental_id_df_sel['Lat'].astype(float).tolist()
# rental_id_df_sel_lon = rental_id_df_sel['Lon'].astype(float).tolist()
# #
# from geopy.geocoders import Nominatim
# #
# address_ls = []
# #
# def geocoding_reverse(lat_lng_str): 
#     # Nomination : 지리적 좌표를 주소로 변환
#     geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
#     # reverse : 위도와 경도를 입력하면 그 위치의 주소를 반환
#     address = geolocoder.reverse(lat_lng_str)
#     #
#     return address
#     #
# #   
# for i,k in zip(rental_id_df_sel_lat, rental_id_df_sel_har):
#     address = geocoding_reverse(f'{i}, {k}')
#     address_ls.append(address)
#     print(address)
#     #
# #
# addr_GeoPy = pd.DataFrame(address_ls)
# addr_GeoPy.to_csv('data/addr_GeoPy.csv', encoding='cp949', index=False)

In [ ]:
# GeoPy롤 위도 경도 값으로 대여소 주소를 가져옴
# 이유는 기존 데이터의 주소가 신주소로 되어 있어,
# 구, 동 구분이 명확하지 않아 구주소로 변환하기 위함
filename0 = r"data/addr_GeoPy.csv" #파일명 작성
data = pd.read_csv(filename0,encoding='cp949')       
local_df = pd.DataFrame(data)
local_df.columns = ['Id','LocalName', 'LocalNum']
# 필터링 라이브러리르 이용하여 GeoPy 데이터에서 구와 동을 추출
local = mp.Filter(local_df)
local_df = local.dong_filter
local_df = local.gu_filter
local_df = local.simplify_dong
local_df = local_df[local_df['LocalName'] != 'Soul Buoy']
local_df = local_df[['Id','Gu','Dong2']]
local_df.rename(columns = {'Dong2':'Dong'}, inplace = True)
import gc
def get_chunks(df, chunk_size):
    num_chunks = len(df) // chunk_size + int(len(df) % chunk_size != 0)
    for i in range(num_chunks):
        # yield 구문은 제너레이터를 리턴합니다.
        # 제너레이터는 이터레이터를 생성해주는 함수입니다.
        # yield 구문을 사용하면 함수는 제너레이터가 되며, 이터레이터처럼 동작합니다.
        # yield 구문을 사용하면 함수는 호출된 위치의 상태를 기억하고 다음 호출 시 이어서 실행합니다.
        # 일반적으로 return 구문은 함수를 종료하고 값을 반환하지만, 
        # yield 구문은 값을 반환한 후 일시정지하고 다음 호출 시 이어서 실행합니다.    
        # 이것은 메모리 측면에서 효율적입니다.
        # for 문의 모든 값을 한번에 return 하는 것이 아니라, 
        # yield를 통해 하나씩 반환합니다.
        yield df[i*chunk_size:(i+1)*chunk_size]
        #
    #
#
def merge_and_optimize_chunks(df1, df2, chunk_size):
    chunks = []
    for chunk1 in get_chunks(df1, chunk_size):
        for chunk2 in get_chunks(df2, chunk_size):
            merged_chunk = pd.merge(chunk1, chunk2, left_on='Id', right_on='Id', how='left')
            chunks.append(merged_chunk)
            del merged_chunk, chunk2, chunk1
            gc.collect()
    big_df = pd.concat(chunks, ignore_index=True)
    return big_df
id_gu_dong_df = merge_and_optimize_chunks(local_df, rental_id_df, 100000)
id_gu_dong_df = id_gu_dong_df[['Id','Gu','Dong']]

#### 아래 파일은 MiniPFinal-2 에서 id_gu_dong_etc_change 로 수정 되어 사용됩니다.
###### 수정된 이유는 폐쇠된 대여소 가 etc 로 찍혀 80 여개의 정보가 누락되어 수기로 채워 넣었기 때문입니다.

In [ ]:
id_gu_dong_df.to_csv('data/id_gu_dong.csv', index=False, encoding='cp949')